# 例題


宮ヶ瀬ダム流域（流域面積100.465038941$\rm{km^2}$)を対象にテスト計算を実施する。

雨量データは'rain.csv'、ダム流入量は'宮ヶ瀬ダム.csv'を使用する。

各種パラメータは以下のとおりとする。

 - $dt$ : 1分
 - $k:40.3$
 - $p:0.5$
 - $R_{sa}:100[\rm{mm}]$
 - $Q_b: 2.0[\rm{m^3/s}]$
 - $f_1: 0.5$
 - $T_l: 0[\rm{hr}]$

In [1]:
import modelStorageFunction as model
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('rain.csv', index_col='date', parse_dates=True)
df

,rainfall
date,
2019-10-11 09:00:00,0.000000
2019-10-11 09:30:00,0.000000
2019-10-11 10:00:00,0.000000
2019-10-11 10:30:00,0.132632
2019-10-11 11:00:00,0.090526
...,...
2019-10-14 06:30:00,0.202105
2019-10-14 07:00:00,0.235789
2019-10-14 07:30:00,0.231579


## 流出高の計算

In [3]:
t = df.index - df.index[0]
time = t.total_seconds()/3600

rain = df['rainfall'].values

dt = float(60/3600) #[hr]
k, p = float(40.3), float(0.5)
Rsa = float(100)

tout, ql1, ql2 = model.calql(time, rain, dt, k, p, Rsa)

## 流出量の計算

In [4]:
Qb = float(2.0)
A = float(100.465038941) #[km^2]
f1 = float(0.5)

Ql = model.calQall(ql1, ql2, f1, A, Qb)

## 遅れ時間のシフト 
省略

## 解析結果を出力

In [5]:
toutr = [df.index[0] + pd.Timedelta(hours = tp) for tp in tout]
dfout = pd.DataFrame({'Q':Ql},index=toutr)
dfout

,Q
2019-10-11 09:00:00,2.000000
2019-10-11 09:01:00,2.000000
2019-10-11 09:02:00,2.000000
2019-10-11 09:03:00,2.000000
2019-10-11 09:04:00,2.000000
...,...
2019-10-14 08:25:00,30.056323
2019-10-14 08:26:00,30.035787
2019-10-14 08:27:00,30.015276
2019-10-14 08:28:00,29.994789


In [7]:
d = dfout.to_csv('calout.csv', index_label='date')